In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy import stats
import statsmodels.api as sm

failure_stresses = pd.DataFrame(
    [
        [2.247, 2.640, 2.842, 2.908, 3.099, 3.126, 3.245, 3.328, 3.355, 3.383, 3.572, 3.581, 3.681],
        [1.901, 2.132, 2.203, 2.228, 2.257, 2.35, 2.361, 2.396, 2.397, 2.445, 2.454, 2.454, 2.474],
        [1.312, 1.314, 1.479, 1.552, 1.700, 1.803, 1.861, 1.865, 1.944, 1.958, 1.966, 1.997, 2.006],
        [1.339, 1.434, 1.549, 1.574, 1.589, 1.613, 1.746, 1.753, 1.764, 1.807, 1.812, 1.840, 1.852],
    ],    
    index=pd.Series([1, 10, 20, 50], name='Length (mm)'))

In [2]:
with open('p2_data.tex', 'w') as f:
    f.write(failure_stresses.reset_index().to_latex(index=False))
failure_stresses

,0,1,2,3,4,5,6,7,8,9,10,11,12
Length (mm),,,,,,,,,,,,,
1,2.247,2.640,2.842,2.908,3.099,3.126,3.245,3.328,3.355,3.383,3.572,3.581,3.681
10,1.901,2.132,2.203,2.228,2.257,2.350,2.361,2.396,2.397,2.445,2.454,2.454,2.474
20,1.312,1.314,1.479,1.552,1.700,1.803,1.861,1.865,1.944,1.958,1.966,1.997,2.006
50,1.339,1.434,1.549,1.574,1.589,1.613,1.746,1.753,1.764,1.807,1.812,1.840,1.852


# Choosing Hyperparameters

We want a Gamma distribution such that the mean is 0.5 and $\mathbb{P}\left([0.05,1]\right) = 0.95$.

In [39]:
from scipy import optimize

def get_gamma_hyperparameters(mean, lower=0.05, upper=1, p=0.95):
    """Fits a gamma distribution with P([lower,upper]) = p with specified mean.
    
    Args:
      mean: Desired expectation of distribution.
      lower: Lower bound of interval.
      upper: Upper bound of interval.
      p: The desired probability of the interval.
      
    Returns:
      The shape and rate parameter of desired gamma distribution.
    """    
    assert 0.05 < mean and mean < upper, 'The mean must between lower and upper.'
    
    def compute_probability(k):
        return (stats.gamma.cdf(upper, mean*k, scale=1/k) -
                stats.gamma.cdf(lower, mean*k, scale=1/k))
    
    scaling_factor = optimize.fsolve(
        lambda k: compute_probability(k) - p, 1)[0]
        
    return mean*scaling_factor, scaling_factor

PRIOR_SHAPE, PRIOR_RATE = get_gamma_hyperparameters(0.5)

print('shape={}, rate={}'.format(PRIOR_SHAPE, PRIOR_RATE))

shape=3.634062529831774, rate=7.268125059663548


In [30]:
stats.gamma.cdf(1, 0.5*7.26812506, scale=1/7.26812506) - stats.gamma.cdf(0.05, 0.5*7.26812506, scale=1/7.26812506)

0.950000000003446